In [ ]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from pymanticore.swift_analysis import HBTData, SOAPData
from stacking_backend import ClusterAnalysisPipeline, BasicPlotter, SummaryPlotter, MassScalingPlotter
from stacking_backend.data import load_pr4_data, PatchExtractor
from pymanticore.cosmology import ra_dec_to_galactic
import h5py

# Initialize the analysis pipeline
pipeline = ClusterAnalysisPipeline()

# Create plotters
basic_plotter = BasicPlotter(pipeline.patch_extractor)
summary_plotter = SummaryPlotter()
mass_plotter = MassScalingPlotter()

In [ ]:
# Example coordinates to plot
examples = [
    {'ra': 194.95, 'dec': 27.98, 'title': 'Coma Cluster'},
    {'ra': 186.69, 'dec': 12.83, 'title': 'Virgo Cluster'},
    {'ra': 23.40, 'dec': 30.66, 'title': 'Perseus Cluster'},
    {'ra': 120.0, 'dec': 0.0, 'title': 'Random Position'},
    {'ra': 168.1, 'dec':  40.45, 'title': "test"}
]

for i, example in enumerate(examples):
    print(f"\n📊 Example {i+1}: {example['title']}")
    
    fig, (ax1, ax2), patch_data, mask_patch = basic_plotter.plot_patch(
        ra=example['ra'],
        dec=example['dec'],
        patch_size_deg=8.0,
        npix=128,
        title=example['title'],
        show_center=True,
        show_grid=True
    )
    
    print(f"Patch shape: {patch_data.shape}")

### Load MCXC cluster catalog

In [ ]:
fname = "/cosma7/data/dp004/rttw52/Manticore/observational_data/mcxc_2_clusters/mcxc_clusters.hdf5"

with h5py.File(fname, "r") as f:
    _mcxc_data = f["all_clusters"][...]

#print(_mcxc_data.dtype)
gal_l, gal_b = ra_dec_to_galactic(np.degrees(_mcxc_data["phi"]), np.degrees(_mcxc_data["theta"]))
mcxc_data = {"gal_l": gal_l, "gal_b": gal_b, "z": _mcxc_data["z"], "m500": _mcxc_data["M500"]*1e14, "r500": _mcxc_data["R500"]}

### Load eROSITA cluster catalog

In [ ]:
fname = "/cosma7/data/dp004/rttw52/Manticore/observational_data/erosita_clusters/erosita_clusters.hdf5"

with h5py.File(fname, "r") as f:
    _eros_data = f["all_clusters"][...]

#print(_mcxc_data.dtype)
gal_l, gal_b = ra_dec_to_galactic(np.degrees(_eros_data["phi"]), np.degrees(_eros_data["theta"]))
eros_data = {"gal_l": gal_l, "gal_b": gal_b, "z": _eros_data["z"], "m500": _eros_data["M500"]*1e14, "r500": _eros_data["R500"]}

## Load Manticore posterior clusters

In [ ]:
import numpy as np
import h5py
from astropy.cosmology import Planck18, z_at_value
import astropy.units as u

fname = "/cosma7/data/dp004/rttw52/Manticore/new_analysis/clusters/temp/output/simplified_clusters.h5"

with h5py.File(fname, "r") as f:
    distances_mpc = f["clusters/median_properties/dist"][...]
    
    # Convert comoving distance to redshift using Planck18 cosmology
    redshifts = []
    for dist in distances_mpc:
        z = z_at_value(Planck18.comoving_distance, dist * u.Mpc)
        redshifts.append(z)
    
    manticore_data = {"gal_l": f["clusters/median_properties/gal_l"][...],
                      "gal_b": f["clusters/median_properties/gal_b"][...],
                      "m500": f["clusters/median_properties/SO_500_crit_TotalMass"][...],
                      "r500": f["clusters/median_properties/SO_500_crit_SORadius"][...],
                      "dist": distances_mpc,
                      "z": np.array(redshifts)}

In [ ]:
def plot_halo_overview(halo_data, z_min=None, z_max=None, figsize=(15, 10), cat_name=""):
    """
    Plot overview of cluster data.
    
    Parameters
    ----------
    halo_data : dict
        Dictionary containing cluster data with keys: 'gal_l', 'gal_b', 'z', 'm500', 'r500'
    z_min : float, optional
        Minimum redshift to include in plots
    z_max : float, optional
        Maximum redshift to include in plots
    figsize : tuple, optional
        Figure size (width, height)
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Apply redshift filter if specified
    if z_min is not None or z_max is not None:
        z_mask = np.ones(len(halo_data["z"]), dtype=bool)
        if z_min is not None:
            z_mask &= (halo_data["z"] >= z_min)
        if z_max is not None:
            z_mask &= (halo_data["z"] <= z_max)
        
        # Create filtered data
        filtered_data = {key: np.array(values)[z_mask] for key, values in halo_data.items()}
        title_suffix = f" (z: {z_min or 'min'} - {z_max or 'max'})"
    else:
        filtered_data = halo_data
        title_suffix = ""
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 3, figsize=figsize)
    fig.suptitle(f'{cat_name} Clusters Overview{title_suffix}', fontsize=16, fontweight='bold')

    # 1. Sky distribution in galactic coordinates
    ax1 = axes[0, 0]
    scatter = ax1.scatter(filtered_data["gal_l"], filtered_data["gal_b"], 
                         c=filtered_data["z"], cmap='viridis', alpha=0.7, s=20)
    ax1.set_xlabel('Galactic Longitude (deg)')
    ax1.set_ylabel('Galactic Latitude (deg)')
    ax1.set_title('Sky Distribution (colored by redshift)')
    ax1.set_xlim(0, 360)
    ax1.set_ylim(-90, 90)
    plt.colorbar(scatter, ax=ax1, label='Redshift')

    # 2. Redshift distribution
    ax2 = axes[0, 1]
    ax2.hist(filtered_data["z"], bins=30, alpha=0.7, edgecolor='black')
    ax2.set_xlabel('Redshift')
    ax2.set_ylabel('Number of Clusters')
    ax2.set_title('Redshift Distribution')
    ax2.axvline(np.median(filtered_data["z"]), color='red', linestyle='--', 
               label=f'Median: {np.median(filtered_data["z"]):.3f}')
    ax2.legend()

    # 3. Mass distribution
    ax3 = axes[0, 2]
    ax3.hist(np.log10(filtered_data["m500"]), bins=30, alpha=0.7, edgecolor='black')
    ax3.set_xlabel('log₁₀(M₅₀₀) [M☉]')
    ax3.set_ylabel('Number of Clusters')
    ax3.set_title('Mass Distribution')
    ax3.axvline(np.log10(np.median(filtered_data["m500"])), color='red', linestyle='--',
               label=f'Median: {np.median(filtered_data["m500"]):.1e} M☉')
    ax3.legend()

    # 4. R500 distribution
    ax4 = axes[1, 0]
    ax4.hist(filtered_data["r500"], bins=30, alpha=0.7, edgecolor='black')
    ax4.set_xlabel('R₅₀₀ [Mpc]')
    ax4.set_ylabel('Number of Clusters')
    ax4.set_title('R₅₀₀ Distribution')
    ax4.axvline(np.median(filtered_data["r500"]), color='red', linestyle='--',
               label=f'Median: {np.median(filtered_data["r500"]):.2f} Mpc')
    ax4.legend()

    # 5. Mass vs Redshift
    ax5 = axes[1, 1]
    scatter2 = ax5.scatter(filtered_data["z"], np.log10(filtered_data["m500"]), 
                          c=filtered_data["r500"], cmap='plasma', alpha=0.7, s=20)
    ax5.set_xlabel('Redshift')
    ax5.set_ylabel('log₁₀(M₅₀₀) [M☉]')
    ax5.set_title('Mass vs Redshift (colored by R₅₀₀)')
    plt.colorbar(scatter2, ax=ax5, label='R₅₀₀ [Mpc]')

    # 6. Data summary statistics
    ax6 = axes[1, 2]
    ax6.axis('off')
    
    n_total = len(halo_data["z"])
    n_filtered = len(filtered_data["z"])
    
    stats_text = f"""Data Summary:
Total Clusters: {n_total:,}
Filtered Clusters: {n_filtered:,}
Filter: z ∈ [{z_min or 'min'}, {z_max or 'max'}]

Redshift:
  Range: {np.min(filtered_data["z"]):.3f} - {np.max(filtered_data["z"]):.3f}
  Mean: {np.mean(filtered_data["z"]):.3f}
  Std: {np.std(filtered_data["z"]):.3f}

Mass (M₅₀₀):
  Range: {np.min(filtered_data["m500"]):.1e} - {np.max(filtered_data["m500"]):.1e} M☉
  Mean: {np.mean(filtered_data["m500"]):.1e} M☉
  
R₅₀₀:
  Range: {np.min(filtered_data["r500"]):.2f} - {np.max(filtered_data["r500"]):.2f} Mpc
  Mean: {np.mean(filtered_data["r500"]):.2f} Mpc

Sky Coverage:
  Gal_l: {np.min(filtered_data["gal_l"]):.1f}° - {np.max(filtered_data["gal_l"]):.1f}°
  Gal_b: {np.min(filtered_data["gal_b"]):.1f}° - {np.max(filtered_data["gal_b"]):.1f}°"""

    ax6.text(0.05, 0.95, stats_text, transform=ax6.transAxes, fontsize=10,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

    plt.tight_layout()
    plt.show()

In [ ]:
# plot_mcxc_overview(mcxc_data)  # Plot all data
plot_halo_overview(manticore_data, z_min=0.001, z_max=0.1, cat_name="Manticore")

In [ ]:
plot_halo_overview(eros_data, z_min=0.001, z_max=0.1, cat_name="Manticore")

In [ ]:
l_min = 20
l_max = 80

def generate_random_halos(n_halos):
    """Generate random positions for null test"""
    random_halos = []
    for i in range(n_halos):
        lon = np.random.uniform(0, 360)
        lat = np.random.uniform(-l_max, l_max)
        if abs(lat) > l_min:
            r200_deg = np.random.lognormal(np.log(1.0), 0.3)
            redshift = np.random.normal(0.05, 0.02)
            random_halos.append((lon, lat, r200_deg, max(0.01, redshift)))
    return random_halos

In [ ]:
# Updated notebook cell for the new analysis
mass_bins = [
    (0.6e14, 0.8e14),
    (0.8e14, 1e14),
    (1e14, 2e14),
    (2e14, 4e14),
    (4e14, 6e14),
]

inner_r200_factor=1.0
outer_r200_factor=3.0
patch_size_deg=20.0
npix=256
z_min = 0.005
z_max = 0.05
b_min = 10
b_max = 80

results_dict = {}

for i, (mass_min, mass_max) in enumerate(mass_bins):
    mass_label = f"{mass_min:.1e}-{mass_max:.1e}"
    print(f"\n📊 Mass bin {i+1}/{len(mass_bins)}: {mass_label} M☉")

    coord_list = np.array([[eros_data["gal_l"][i], eros_data["gal_b"][i], eros_data["r500"][i], eros_data["z"][i]] 
                          for i in range(len(eros_data["z"])) 
                          if eros_data['z'][i] > z_min and eros_data['z'][i] < z_max 
                          and np.abs(eros_data['gal_b'][i]) > b_min and np.abs(eros_data['gal_b'][i]) < b_max 
                          and eros_data['m500'][i] >= mass_min and eros_data['m500'][i] <= mass_max])

    results = pipeline.run_individual_r200_analysis_with_validation(
        coord_list=coord_list,
        inner_r200_factor=inner_r200_factor,
        outer_r200_factor=outer_r200_factor,
        patch_size_deg=patch_size_deg,
        npix=npix,
        min_coverage=0.8,
        n_radial_bins=30,
        run_null_tests=True,
        n_bootstrap=500,
        n_random=200
    )
    
    results['mass_range'] = (mass_min, mass_max)
    results['mass_label'] = mass_label
    results['mass_center'] = np.sqrt(mass_min * mass_max)
    
    results_dict[mass_label] = results
    
    if results['null_significance']:
        print(f"   ✅ Analysis complete: {results['significance']:.1f}σ detection, {results['null_significance']:.1f}σ vs null")
    else:
        print(f"   ✅ Analysis complete: {results['significance']:.1f}σ from {results['n_measurements']} clusters")

In [ ]:
fig1, ax1 = summary_plotter.plot_mass_range_profiles(results_dict)

In [ ]:
for mass_range, results in results_dict.items():

    inner_radius_deg = results['r200_median'] * inner_r200_factor  # inner_r200_factor
    outer_radius_deg = results['r200_median'] *  outer_r200_factor # outer_r200_factor
    
    # Or use the actual ranges if you want to show the variation
    inner_radii_range = results['inner_radii_range']  # (min, max)
    outer_radii_range = results['outer_radii_range']  # (min, max)
    
    # Plot the stacked patch
    basic_plotter.plot_stacked_patch(
        stacked_patch=results['stacked_patch'],
        patch_size_deg=results['patch_size_deg'],
        title="Stacked Y-Map with Individual R200 Apertures",
        show_apertures=True,
        inner_radius_deg=inner_radius_deg,  # Use median values
        outer_radius_deg=outer_radius_deg
    )

In [ ]:
fig, ax, fit_results = mass_plotter.plot_y_mass_scaling(
    results_dict, 
    title="kSZ Detection: Y-M Scaling",
    show_theory=True,
    fit_scaling=True
)

# Check fit results
if fit_results:
    print(f"Your slope: {fit_results['fitted_slope']:.2f}")
    print(f"Theory slope: {fit_results['theory_slope']:.2f}")